In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
path_to_swimAD2 = '/Users/kiwi/Home/2022/2학기/중급물리실험 2/swimAD2 230103'
if path_to_swimAD2 not in sys.path:
    sys.path.append(path_to_swimAD2)

import swimAD2 as ad2
from ctypes import *
import time

ad2.disconnect()    # close all instances before connecting
alice = ad2.connect(0)       # connect to first(0th) ad2

In [ ]:
# Easiest to use: ad2.basic_measure()
t, v1, v2 = ad2.basic_measure(
    # device
    hdwf = alice,

    # wavegen settings
    frequency = 10e3,               # max 10 MHz
    amplitude = 1,                  # max 5 V
    sample_rate = 1000e3,           # max 100 MHz

    # oscilloscope settings
    range1 = 2,                     # Oscilloscope Channel 1 voltage range, max 25 V
    range2 = 2,                     # Oscilloscope Channel 2

    # additional parameters
    signal_shape = ad2.dwfc.funcSine,   # output signal shape: sine, square, triangle, rampup, rampdown, noise
    offset = 0,                                 # signal voltage offset, set stabilizing time > 2 sec if not 0
    phase = 0,                                  # signal starting phase
    symmetry = 50,                              # signal symmetry shape (default 50)

    sample_size = 8192,         # max 8192, sample time = sample_size / sample_rate
    stabilizing_time = 0.05,    # stabilize oscilloscope after initial output
)

In [ ]:
# more detailed control:
# config_output() - start_output() - (stabilizing time if needed) - config_input() - measure() - end_output() - reset_output()

ad2.config_output(
    # wavegen settings
    frequency = 10e3,                              # max 10 MHz
    amplitude = 1,                              # max 5 V
    # signal_shape = signal_shapes['sine'],   # output signal shape: funcSine, funcSquare, funcTriangle, funcRampUp, funcRampDown, funcNoise
    # offset = 0,                             # signal voltage offset, set stabilizing time > 2 sec if not 0
    # phase = 0,                              # signal phase
    # symmetry = 50,                          # signal symmetry shape
    channel = c_int(0)                      # channel 0 / 1 (or -1 for all channels) 
    )

ad2.start_output(channel = c_int(0))

time.sleep(0.1)

ad2.config_input(
    # oscilloscope settings
    range1 = 2,                     # Oscilloscope Channel 1 voltage range, max 25 V
    range2 = 2,                     # Oscilloscope Channel 2
    sample_rate = 1000e3,                # max 100 MHz
    sample_size = 8192,         # max 8192, sample time = sample_size / sample_rate
)

t, v1, v2 = ad2.measure()

ad2.end_output(channel = c_int(0))
ad2.reset_output(channel = c_int(0))

In [ ]:
amplitudes = [1, 2, 3]
frequencies = np.linspace(0.1e6, 0.5e6, 500)[::]
offset = -3

In [ ]:
DATA = []

for a in amplitudes[::]:
    data = []

    ad2.config_output(frequency=0, amplitude=0, offset=offset, channel=c_int(1))
    ad2.start_output(channel=c_int(1))

    time.sleep(0.2)

    for f in tqdm(frequencies[::]):
        
        ad2.config_input(range1=2, range2=a*2, sample_rate=512*f)
        ad2.config_output(frequency=f, amplitude=a, channel=c_int(0))
        ad2.start_output(channel=c_int(0))

        t, v1, v2 = ad2.measure()

        ad2.stop_output(channel=c_int(0))
        
        nv1, nv2 = v1 - v1.mean(), v2 - v2.mean()

        dt = np.array([t, v1, v2, nv1, nv2])
        data.append(dt)
    
    DATA.append(data)
    
    ad2.reset_output()

DATA = np.array(DATA)

RATIOS, PHASES = [], []
for data in DATA:
    ratios, phases = [], []
    for dt in data[::]:
        t, v1, v2, nv1, nv2 = dt

        # plt.plot(t, nv1, '.')
        # plt.plot(t, nv2, '.')
        # plt.ylim(-6, 6)
        # plt.show()

        v1_amplitude = np.sqrt(2) * (nv1**2).mean()**(1/2)
        v2_amplitude = np.sqrt(2) * (nv2**2).mean()**(1/2)
        ratio = v2_amplitude / v1_amplitude
        ratios.append(ratio)

        product = nv1 * nv2
        product_amplitude = v1_amplitude * v2_amplitude / 2
        cos_phase = product.mean() / product_amplitude
        phase = np.arccos(cos_phase if abs(cos_phase) < 1 else 1)
        phases.append(phase)

    RATIOS.append(ratios)
    PHASES.append(phases)

RATIOS, PHASES = np.array(RATIOS), np.array(PHASES)

In [ ]:
for i, a in enumerate(amplitudes):
    ratios = RATIOS[i]
    plt.plot(frequencies, ratios*a, '.', markersize=1)